In [1]:
# pip install pulp

In [2]:
import pulp
from pulp import *

import numpy as np
import pandas as pd

import datetime
import matplotlib.pyplot as plt

## Load Data

In [3]:
path = 'CSV_Files_Food_Delivery/'

In [4]:
df_distances = pd.read_csv(path + 'distances.csv')

df_ordersA = pd.read_csv(path + 'part1_ordersA.csv')
df_ordersB = pd.read_csv(path + 'part1_ordersB.csv')

In [5]:
df_distances.head()

,origin,destination,distance
0,Scarborough (Malvern / Rouge River),Scarborough (Rouge Hill / Port Union / Highlan...,3.931478
1,Scarborough (Malvern / Rouge River),Scarborough (Guildwood / Morningside / Ellesmere),4.864191
2,Scarborough (Malvern / Rouge River),Scarborough (Woburn),4.778347
3,Scarborough (Malvern / Rouge River),Scarborough (Cedarbrae),6.009861
4,Scarborough (Malvern / Rouge River),Scarborough (Eglinton),7.876162


In [6]:
df_ordersA

,restaurant,customer
0,Downtown Toronto (Underground city),Downtown Toronto (Central Bay Street)


In [7]:
df_ordersB

,restaurant,customer
0,Downtown Toronto (Central Bay Street),Downtown Toronto (Underground city)
1,Etobicoke Northwest (Clairville / Humberwood /...,Etobicoke (South Steeles / Silverstone / Humbe...
2,York (Cedarvale),Central Toronto (The Annex / North Midtown / Y...
3,Downtown Toronto (Central Bay Street),Downtown Toronto (Richmond / Adelaide / King)
4,Downtown Toronto (Richmond / Adelaide / King),Downtown Toronto (St. James Town / Cabbagetown)


## Sets

In [8]:
starting_location = 'Downtown Toronto (Rosedale)'

In [9]:
df = df_ordersB

In [10]:
# Get start and end locations
start_locations = list(df['restaurant'].unique()) + list(df['customer'].unique())
end_locations = list(df['restaurant'].unique()) + list(df['customer'].unique())

start_locations.append('Downtown Toronto (Rosedale)')

start_locations = list(dict.fromkeys(start_locations))
end_locations = list(dict.fromkeys(end_locations))

# Calculate the number of stops 
num_stops = np.arange(0, len(end_locations)).tolist()

In [11]:
# Get unique Restraurants and Customers
restaurants = df['restaurant'].unique().tolist()
customers = df['customer'].unique().tolist()

In [12]:
restaurants

['Downtown Toronto (Central Bay Street)',
 'Etobicoke Northwest (Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale)',
 'York (Cedarvale)',
 'Downtown Toronto (Richmond / Adelaide / King)']

In [13]:
customers

['Downtown Toronto (Underground city)',
 'Etobicoke (South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens)',
 'Central Toronto (The Annex / North Midtown / Yorkville)',
 'Downtown Toronto (Richmond / Adelaide / King)',
 'Downtown Toronto (St. James Town / Cabbagetown)']

In [14]:
list_order = []
for index, row in df.iterrows():
    list_order.append([row["restaurant"], row["customer"]])
print(list_order)

[['Downtown Toronto (Central Bay Street)', 'Downtown Toronto (Underground city)'], ['Etobicoke Northwest (Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale)', 'Etobicoke (South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens)'], ['York (Cedarvale)', 'Central Toronto (The Annex / North Midtown / Yorkville)'], ['Downtown Toronto (Central Bay Street)', 'Downtown Toronto (Richmond / Adelaide / King)'], ['Downtown Toronto (Richmond / Adelaide / King)', 'Downtown Toronto (St. James Town / Cabbagetown)']]


In [15]:
travel_distance = {}

# Distance between start location and end location
for i in start_locations:
    for j in end_locations: 
        if i == j:
            travel_distance[(i,j)] = 0 
        else:
            travel_distance[(i,j)] = float(df_distances[(df_distances['origin'] == i) & 
                                                          (df_distances['destination'] == j)]['distance'])

In [16]:
travel_distance

{('Downtown Toronto (Central Bay Street)',
  'Downtown Toronto (Central Bay Street)'): 0,
 ('Downtown Toronto (Central Bay Street)',
  'Etobicoke Northwest (Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale)'): 17.693277187517552,
 ('Downtown Toronto (Central Bay Street)',
  'York (Cedarvale)'): 5.306326643787583,
 ('Downtown Toronto (Central Bay Street)',
  'Downtown Toronto (Richmond / Adelaide / King)'): 0.7867106676349092,
 ('Downtown Toronto (Central Bay Street)',
  'Downtown Toronto (Underground city)'): 0.8541546574657076,
 ('Downtown Toronto (Central Bay Street)',
  'Etobicoke (South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens)'): 18.86304834675905,
 ('Downtown Toronto (Central Bay Street)',
  'Central Toronto (The Annex / North Midtown / Yorkville)'): 2.374732498057164,
 ('Downtown Toronto (Central Bay Street)',
  'Downtown Toronto (St. James Tow

## Model Setup

In [24]:
num_stops

[0, 1, 2, 3, 4, 5, 6, 7]

In [26]:
end_locations

['Downtown Toronto (Central Bay Street)',
 'Etobicoke Northwest (Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale)',
 'York (Cedarvale)',
 'Downtown Toronto (Richmond / Adelaide / King)',
 'Downtown Toronto (Underground city)',
 'Etobicoke (South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens)',
 'Central Toronto (The Annex / North Midtown / Yorkville)',
 'Downtown Toronto (St. James Town / Cabbagetown)']

In [17]:
model = LpProblem(name = 'Model', sense = LpMinimize)

xVar = LpVariable.dict('x', (start_locations, end_locations, num_stops), cat = LpBinary)

In [18]:
obj = lpSum([travel_distance[(i,j)] * xVar[(i,j,t)] for i in start_locations for j in end_locations for t in num_stops])
model += obj

## Constraints

In [19]:
# 1. Convervation of flow
for t in num_stops[:-1]:
    for j in end_locations:
        model += (lpSum([xVar[(i,j,t)] for i in start_locations]) == lpSum([xVar[(j,k,t+1)] for k in end_locations]))

# 2. Every location is visited once
for j in end_locations: 
    model += lpSum([xVar[(i,j,t)] for i in start_locations for t in num_stops]) == 1
    
# 3. First node has an outflow of 1, others 0
for i in start_locations:
    if i == starting_location:
        model += lpSum([xVar[(i,j,0)] for j in end_locations]) == 1
    else: 
        model += lpSum([xVar[(i,j,0)] for j in end_locations]) == 0

# 4. Visit restaurant before customer
def restaurant_customer_match(customer):
    for order in list_order:
        if customer == order[1]:
            return order[0]

for t in num_stops: 
    for j in end_locations: 
        if j in customers: 
            model += pulp.lpSum( [xVar[i,j,t] for i in start_locations]) <= pulp.lpSum([xVar[i, restaurant_customer_match(j),t_] for i in start_locations for t_ in num_stops[:t]])

## Solve

In [20]:
# Solve the model
model.solve()
print("Status:", LpStatus[model.status])

Status: Optimal


In [21]:
# Total Distance
total_distance = pulp.value(model.objective)
print("Total Distance: " , total_distance)

Total Distance:  33.875942274546034


In [45]:
# Print the results
path=[]
for t in num_stops:
    print(f'step:{t}\n')
    for i in start_locations:
         for j in end_locations:
                if (xVar[(i,j,t)].varValue == 1) :
                    print(f"TRAVEL FROM {i}  TO {j}")
                    
                    if i not in path:
                        path.append(i)
                    if j not in path:
                        path.append(j)

step:0

TRAVEL FROM Downtown Toronto (Rosedale)  TO Downtown Toronto (Central Bay Street)
step:1

TRAVEL FROM Downtown Toronto (Central Bay Street)  TO Downtown Toronto (Richmond / Adelaide / King)
step:2

TRAVEL FROM Downtown Toronto (Richmond / Adelaide / King)  TO Downtown Toronto (Underground city)
step:3

TRAVEL FROM Downtown Toronto (Underground city)  TO Downtown Toronto (St. James Town / Cabbagetown)
step:4

TRAVEL FROM Downtown Toronto (St. James Town / Cabbagetown)  TO York (Cedarvale)
step:5

TRAVEL FROM York (Cedarvale)  TO Central Toronto (The Annex / North Midtown / Yorkville)
step:6

TRAVEL FROM Central Toronto (The Annex / North Midtown / Yorkville)  TO Etobicoke Northwest (Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale)
step:7

TRAVEL FROM Etobicoke Northwest (Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale)  TO Etobicoke (South Steel